In [ ]:
# 构建一个表示图中节点间相连度的邻接矩阵（Adjacency Matrix）
# 记录所有图节点与其他节点的相连度（0-1表示，默认为0[不相连]，根据相连距离归一化表示，值越大表示越接近）
import numpy as np
import pandas as pd

dataset_name="BRT"
file_path = f"../data/{dataset_name}/{dataset_name}.csv"
# file_path = "../data/PeMS08/PEMS08.csv"
# file_path = "../data/PeMS04/PEMS04.csv"

In [ ]:

def get_similarity_data(file_path):
    data = pd.read_csv(file_path)
    data = data.to_numpy()
    data_index = np.array(data[:,:2],dtype=np.int32)
    data_cost = np.array(data[:,2],dtype=np.float32)
    print(data_index.shape)
    print(data_cost.shape)

    # 使用归一化公式将原始距离转换为0到1之间的值，并反向表示接近度。通常情况下，距离与接近度成反比
    max_dis = np.max(data_cost)
    min_dis = np.min(data_cost)
    print("max_dis = ", max_dis)
    print("min_dis = ", min_dis)
    normalized = 1. - (data_cost - min_dis) / (max_dis - min_dis)
    return data_index, normalized
    # return data_index, data_cost

data_index, normalized = get_similarity_data(file_path=file_path)
print(data_index[:5])
print(normalized[:5])

In [ ]:
def get_np_adj(num_vertex, data_index, data_cost):
    adj_index = np.zeros((num_vertex*num_vertex, 2), dtype=np.int32)
    adj_cost = np.zeros((num_vertex*num_vertex, 1), dtype=np.float32)
    print(adj_index.shape)
    print(adj_cost.shape)
    for i in range(adj_index.shape[0]):
        adj_index[i,0]=i // num_vertex
        adj_index[i,1]=i % num_vertex
        if adj_index[i,0] == adj_index[i,1]:
            adj_cost[i]=1
    # print(adj_index[(num_vertex-10):(num_vertex+10)])
    for i in range(data_index.shape[0]):
        A = data_index[i,0]
        B = data_index[i,1]
        cost = data_cost[i]
        for j in range(adj_index.shape[0]):
            if A==adj_index[j,0] and B==adj_index[j,1]:
                adj_cost[j]=cost
            # elif A==adj_index[j,1] and B==adj_index[j,0]: # 双向图
            #     adj_cost[j]=cost

    return adj_index, adj_cost

adj_index, adj_cost = get_np_adj(45,data_index,normalized) # BRT
# adj_index, adj_cost = get_np_adj(170,data_index,normalized) # PEMS07
# adj_index, adj_cost = get_np_adj(170,data_index,normalized) # PEMS08
# adj_index, adj_cost = get_np_adj(307,data_index,normalized) # PEMS04


In [ ]:
# save
# 写入adj.txt文件
# adj_file_path = '../data/PeMS04/PeMS04_Adj.txt'
# adj_file_path = '../data/PeMS08/PeMS08_Adj.txt'
adj_file_path = f'../data/{dataset_name}/Adj_{dataset_name}.txt'
lines = []
for index, cost in zip(adj_index, adj_cost):
    line = f"{index[0]} {index[1]} {cost[0]:.6f}"
    lines.append(line)
with open(adj_file_path, 'w') as f:
    f.write("\n".join(lines))
